In [1]:
from pymongo import MongoClient
import pandas as pd

connection=MongoClient("mongodb://192.168.0.18:27017/crawler.contents")

db=connection.get_database()
articles = pd.DataFrame(list(db.contents.find()))


articles = articles.drop(columns=['visited','created_at','contentType','date','icon'])
articles.head()

,_id,title,summary,url,source,alternateImageUrl,body,publishedAt,classes_target,classes_body,classes_summary,classes_title,summary_textrank
0,623d62126e887c1f5ed1a34e,"Risk currencies recover from Friday carnage, d...",The Australian dollar and other riskier curren...,https://www.reuters.com/article/uk-global-fore...,reuters.com,https://static.reuters.com/resources/r/?m=02&d...,"[By Reuters Staff, 3 MIN READ, TOKYO (Reuters)...",2021-03-01T01:10:24Z,"[GBP, JPY, NZD, EUR, AUD, USD, INDEX]","[GBP, JPY, NZD, EUR, AUD, USD, INDEX]","[AUD, USD]",[USD],TOKYO (Reuters) - The Australian dollar and ot...
1,623d62126e887c1f5ed1a350,"Taiwan punishes Deutsche Bank, others in curre...",Taiwan's central bank said on Sunday it had ba...,https://www.reuters.com/article/uk-taiwan-fore...,reuters.com,https://static.reuters.com/resources/r/?m=02&d...,"[By Reuters Staff, 3 MIN READ, TAIPEI (Reuters...",2021-02-07T06:44:13Z,"[TWD, USD]","[TWD, USD]","[TWD, USD]",[],TAIPEI (Reuters) - Taiwan’s central bank said ...
2,623d62126e887c1f5ed1a34c,"BoE hikes, Fed pivots, ECB rolls slow as pande...",(Reuters) -Britain became the first G7 economy...,https://www.reuters.com/article/global-cenbank...,reuters.com,https://www.reuters.com/resizer/2PugZXYjkc1RTi...,"[Federal Reserve in Washington, U.S., November...",2021-12-16T14:46:19Z,"[INDEX, EUR]","[INDEX, EUR]",[EUR],[INDEX],REUTERS/Kevin Lamarque/File Photo Dec 16 (Reut...
3,623d62126e887c1f5ed1a351,Dollar extends rebound as U.S. yields rise,The dollar edged higher across the board on Mo...,https://www.reuters.com/article/uk-global-fore...,reuters.com,https://static.reuters.com/resources/r/?m=02&d...,"[By Saqib Iqbal Ahmed, 3 MIN READ, NEW YORK (R...",2021-01-11T01:32:00Z,"[GBP, USD, BTC]","[GBP, USD, BTC]",[USD],[USD],By Saqib Iqbal Ahmed NEW YORK (Reuters) - The...
4,623d62126e887c1f5ed1a352,Dollar recovery pushes sterling below $1.35; U...,The pound was among the risk assets hurt by th...,https://www.reuters.com/article/uk-britain-ste...,reuters.com,https://static.reuters.com/resources/r/?m=02&d...,"[By Elizabeth Howcroft, 3 MIN READ, LONDON (Re...",2021-01-11T09:44:15Z,"[GBP, INDEX, USD, EUR]","[GBP, INDEX, USD, EUR]","[GBP, INDEX, USD]","[GBP, USD]",By Elizabeth Howcroft LONDON (Reuters) - The ...


In [2]:
import torch

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

C:\Users\Igo\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1060 6GB


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, PreTrainedTokenizerFast
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)

In [4]:
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)


In [6]:
import pandas as pd


In [7]:
articles = articles.drop(columns=['classes_target','classes_summary','classes_title'])



In [8]:
articles['text'] = articles['body'].map(lambda x: " ".join(x) if type(x) is list else "")
# summarizer(list(files.get('text')),max_length=120,min_length=30,do_sample=False, batch_size=4),
# fast_tokenizer = PreTrainedTokenizerFast(tokenizer=tokenizer)
def split_by_window_size(tokens):
    start = 0
    window_size = 510
    chunksize = 512
    input_id_chunks =  list(tokens['input_ids'][0].split(chunksize - 2))
    mask_chunks =  list(tokens['attention_mask'][0].split(chunksize - 2))
    for i in range(len(input_id_chunks)):
        # add CLS and SEP tokens to input IDs
        input_id_chunks[i] = torch.cat([
            torch.tensor([101]).to(device), input_id_chunks[i].to(device), torch.tensor([102]).to(device)
        ]).to(device)
        # add attention tokens to attention mask
        mask_chunks[i] = torch.cat([
            torch.tensor([1]).to(device), mask_chunks[i].to(device), torch.tensor([1]).to(device)
        ]).to(device)
        # get required padding length
        pad_len = chunksize - input_id_chunks[i].shape[0]
        # check if tensor length satisfies required chunk size
        if pad_len > 0:
            # if padding length is more than 0, we must add padding
            input_id_chunks[i] = torch.cat([
                input_id_chunks[i], torch.Tensor([0] * pad_len).to(device)
            ]).to(device)
            mask_chunks[i] = torch.cat([
                mask_chunks[i], torch.Tensor([0] * pad_len).to(device)
            ]).to(device)
    input_ids = torch.stack(input_id_chunks).to(device)
    attention_mask = torch.stack(mask_chunks).to(device)

    input_dict = {
        'input_ids': input_ids.long(),
        'attention_mask': attention_mask.int()
    }
    return input_dict

import time
from transformers import SummarizationPipeline

class MyPipeline(SummarizationPipeline):
    def preprocess(self, *args, **kwargs):
        inputs = super().preprocess(*args, **kwargs)
        inputs = split_by_window_size(inputs)
        
        return inputs
summarizer = MyPipeline( model=model, tokenizer=tokenizer, device=0)
s = time.time()
outputs = summarizer(list(articles.get('text')),max_length=120,min_length=30,do_sample=False, batch_size=8)
e = time.time()
print(round(e-s,10))
print(len(outputs))
# with torch.no_grad():
#     inputs = files['text'].map(lambda x: tokenizer(x , return_tensors="pt").to(device) )
#     inputs = inputs.map(lambda tokens: split_by_window_size(tokens))
#     outputs = inputs.map(lambda x: model.generate(**x))


Token indices sequence length is longer than the specified maximum sequence length for this model (1169 > 1024). Running this sequence through the model will result in indexing errors


26788.2259185314
27919


In [9]:
print(outputs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
articles['summary_bert'] = [x['summary_text'] for x in outputs]

In [11]:
for i,x in articles.iterrows():
    db.contents.update_one({'_id':x['_id']},{'$set':{'summary_bert':x['summary_bert']}})